# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [10]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

RANDOM_STATE = 110
ROOT_DIR = "data"

In [11]:
#WorkMode 제거

### 데이터 읽어오기


In [12]:
train_data = pd.read_csv("train_mod.csv")
# train_data = pd.read_csv("shifted_train.csv")
X = train_data.drop(columns=['target'])
y = train_data['target']
X.head()

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,...,Head Clean Position Y Collect Result_Fill2,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,2.5,-90,...,50,91.8,270.0,50,114.612,19.9,7.0,127,1,NaN
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,2.5,-90,...,50,91.8,270.0,50,85.000,19.6,7.0,185,1,0.0
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,12.5,90,...,50,91.8,270.0,50,114.612,19.8,10.0,73,1,NaN
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,12.5,90,...,50,91.8,270.0,50,85.000,19.9,12.0,268,1,0.0
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,2.5,-90,...,50,91.8,270.0,50,85.000,19.7,8.0,121,1,0.0


In [13]:
coordinate_columns = [
    ("HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam", "HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam", "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam", "HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam", "HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam", "HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam", "HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam"),
    ("HEAD Standby Position X Collect Result_Dam", "HEAD Standby Position Y Collect Result_Dam", "HEAD Standby Position Z Collect Result_Dam"),
    ("Head Clean Position X Collect Result_Dam", "Head Clean Position Y Collect Result_Dam", "Head Clean Position Z Collect Result_Dam"),
    ("Head Purge Position X Collect Result_Dam", "Head Purge Position Y Collect Result_Dam", "Head Purge Position Z Collect Result_Dam"),
    ("Head Zero Position X Collect Result_Dam", "Head Zero Position Y Collect Result_Dam", "Head Zero Position Z Collect Result_Dam"),
    
    ("HEAD Standby Position X Collect Result_Fill1", "HEAD Standby Position Y Collect Result_Fill1", "HEAD Standby Position Z Collect Result_Fill1"),
    ("Head Clean Position X Collect Result_Fill1", "Head Clean Position Y Collect Result_Fill1", "Head Clean Position Z Collect Result_Fill1"),
    ("Head Purge Position X Collect Result_Fill1", "Head Purge Position Y Collect Result_Fill1", "Head Purge Position Z Collect Result_Fill1"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1", "HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1", "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1", "HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1", "HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1", "HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1", "HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1"),
    
    ("HEAD Standby Position X Collect Result_Fill2", "HEAD Standby Position Y Collect Result_Fill2", "HEAD Standby Position Z Collect Result_Fill2"),
    ("Head Clean Position X Collect Result_Fill2", "Head Clean Position Y Collect Result_Fill2", "Head Clean Position Z Collect Result_Fill2"),
    ("Head Purge Position X Collect Result_Fill2", "Head Purge Position Y Collect Result_Fill2", "Head Purge Position Z Collect Result_Fill2"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2", "HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2", "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2", "HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2", "HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2", "HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2", "HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2"),
]

cylindrical_columns = [
    ("CURE START POSITION X Collect Result_Dam", "CURE START POSITION Z Collect Result_Dam", "CURE START POSITION Θ Collect Result_Dam"),
    ("CURE STANDBY POSITION X Collect Result_Dam", "CURE STANDBY POSITION Z Collect Result_Dam", "CURE STANDBY POSITION Θ Collect Result_Dam"),
    ("CURE END POSITION X Collect Result_Dam", "CURE END POSITION Z Collect Result_Dam", "CURE END POSITION Θ Collect Result_Dam"),
    ("CURE START POSITION X Collect Result_Fill2", "CURE START POSITION Z Collect Result_Fill2", "CURE START POSITION Θ Collect Result_Fill2"),
    ("CURE STANDBY POSITION X Collect Result_Fill2", "CURE STANDBY POSITION Z Collect Result_Fill2", "CURE STANDBY POSITION Θ Collect Result_Fill2"),
    ("CURE END POSITION X Collect Result_Fill2", "CURE END POSITION Z Collect Result_Fill2", "CURE END POSITION Θ Collect Result_Fill2"),
]

for x_col, y_col, z_col in coordinate_columns:
    
    X[x_col] = np.sqrt(X[x_col]**2 + X[y_col]**2 + X[z_col]**2)
    X[y_col] = np.arctan2(X[y_col], X[x_col])
    X[z_col] = np.arccos(X[z_col] / X[x_col])

    r_col = x_col.replace("X", "r").replace("Y", "θ").replace("Z", "φ")
    theta_col = y_col.replace("X", "r").replace("Y", "θ").replace("Z", "φ")
    phi_col = z_col.replace("X", "r").replace("Y", "θ").replace("Z", "φ")
    
    X.rename(columns={x_col: r_col, y_col: theta_col, z_col: phi_col}, inplace=True)

for x_col, z_col, theta_col in cylindrical_columns:
    
    X[x_col] = np.sqrt(X[x_col]**2 + X[z_col]**2)
    # df[theta_col] = df[theta_col]
    X[z_col] = np.arctan2(X[x_col], X[z_col])
    
    r_col = x_col.replace("X", "r").replace("Z", "φ").replace("Θ", "θ")
    phi_col = z_col.replace("X", "r").replace("Z", "φ").replace("Θ", "θ")
    
    X.rename(columns={x_col: r_col, z_col: phi_col}, inplace=True)

X.head()

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION r Collect Result_Dam,CURE END POSITION φ Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,...,Head Clean Position θ Collect Result_Fill2,Head Clean Position φ Collect Result_Fill2,Head Purge Position r Collect Result_Fill2,Head Purge Position θ Collect Result_Fill2,Head Purge Position φ Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.013020,1.560381,-90,...,0.305771,0.952593,297.549845,0.166484,1.175387,19.9,7.0,127,1,NaN
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.013020,1.560381,-90,...,0.305771,0.952593,287.445647,0.172223,1.270600,19.6,7.0,185,1,0.0
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.078122,1.558298,90,...,0.305771,0.952593,297.549845,0.166484,1.175387,19.8,10.0,73,1,NaN
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.078122,1.558298,90,...,0.305771,0.952593,287.445647,0.172223,1.270600,19.9,12.0,268,1,0.0
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.013020,1.560381,-90,...,0.305771,0.952593,287.445647,0.172223,1.270600,19.7,8.0,121,1,0.0


In [14]:
train_data = pd.concat([X, y], axis=1)
train_data.head()

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION r Collect Result_Dam,CURE END POSITION φ Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,...,Head Clean Position φ Collect Result_Fill2,Head Purge Position r Collect Result_Fill2,Head Purge Position θ Collect Result_Fill2,Head Purge Position φ Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.013020,1.560381,-90,...,0.952593,297.549845,0.166484,1.175387,19.9,7.0,127,1,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.013020,1.560381,-90,...,0.952593,287.445647,0.172223,1.270600,19.6,7.0,185,1,0.0,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.078122,1.558298,90,...,0.952593,297.549845,0.166484,1.175387,19.8,10.0,73,1,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.078122,1.558298,90,...,0.952593,287.445647,0.172223,1.270600,19.9,12.0,268,1,0.0,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.013020,1.560381,-90,...,0.952593,287.445647,0.172223,1.270600,19.7,8.0,121,1,0.0,Normal


In [15]:
train_data.to_csv('train_sph.csv', index=False)

In [16]:
from sklearn.preprocessing import LabelEncoder

non_numeric_columns = X.select_dtypes(include=['object']).columns

encoder = LabelEncoder()
encoded_df = pd.DataFrame(index=X.index)
for column in non_numeric_columns:
    encoded_df[column] = encoder.fit_transform(X[column])

X = X.drop(columns=non_numeric_columns)
X = pd.concat([X, encoded_df], axis=1)
X.head()

,Insp. Seq No._Dam,CURE END POSITION r Collect Result_Dam,CURE END POSITION φ Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE STANDBY POSITION r Collect Result_Dam,CURE STANDBY POSITION φ Collect Result_Dam,CURE STANDBY POSITION Θ Collect Result_Dam,CURE START POSITION r Collect Result_Dam,CURE START POSITION φ Collect Result_Dam,...,Equipment_Fill1,Model.Suffix_Fill1,Workorder_Fill1,Insp Judge Code_Fill1,Wip Line_Fill2,Process Desc._Fill2,Equipment_Fill2,Model.Suffix_Fill2,Workorder_Fill2,Insp Judge Code_Fill2
0,1,240.013020,1.560381,-90,100,1150.487831,1.541686,0,1030.544638,1.538301,...,0,3,657,0,0,0,0,3,657,0
1,1,240.013020,1.560381,-90,70,1150.487831,1.541686,0,1030.544638,1.538301,...,0,3,283,0,0,0,0,3,283,0
2,1,1000.078122,1.558298,90,85,1150.487831,1.541686,0,281.996897,1.452555,...,1,0,589,0,0,0,1,0,589,0
3,1,1000.078122,1.558298,90,70,1150.487831,1.541686,0,281.996897,1.452555,...,1,0,251,0,0,0,1,0,251,0
4,1,240.013020,1.560381,-90,70,1150.487831,1.541686,0,1030.544638,1.538301,...,0,0,142,0,0,0,0,0,142,0


In [81]:
from sklearn.preprocessing import StandardScaler

X_columns = X.columns

scaler = StandardScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X, columns=X_columns)
X

,Insp. Seq No._Dam,CURE END POSITION r Collect Result_Dam,CURE END POSITION φ Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE STANDBY POSITION r Collect Result_Dam,CURE STANDBY POSITION φ Collect Result_Dam,CURE STANDBY POSITION Θ Collect Result_Dam,CURE START POSITION r Collect Result_Dam,CURE START POSITION φ Collect Result_Dam,...,Equipment_Fill1,Model.Suffix_Fill1,Workorder_Fill1,Insp Judge Code_Fill1,Wip Line_Fill2,Process Desc._Fill2,Equipment_Fill2,Model.Suffix_Fill2,Workorder_Fill2,Insp Judge Code_Fill2
0,0.0,-0.786319,0.786319,-0.786319,2.677342,0.0,4.440892e-16,0.0,0.786319,0.786319,...,-0.786443,2.860186,1.908389,0.0,0.0,0.0,-0.786484,2.860186,1.908389,0.0
1,0.0,-0.786319,0.786319,-0.786319,-0.395838,0.0,4.440892e-16,0.0,0.786319,0.786319,...,-0.786443,2.860186,-0.132974,0.0,0.0,0.0,-0.786484,2.860186,-0.132974,0.0
2,0.0,1.271748,-1.271748,1.271748,1.140752,0.0,4.440892e-16,0.0,-1.271748,-1.271748,...,1.271549,-0.373781,1.537232,0.0,0.0,0.0,1.271482,-0.373781,1.537232,0.0
3,0.0,1.271748,-1.271748,1.271748,-0.395838,0.0,4.440892e-16,0.0,-1.271748,-1.271748,...,1.271549,-0.373781,-0.307636,0.0,0.0,0.0,1.271482,-0.373781,-0.307636,0.0
4,0.0,-0.786319,0.786319,-0.786319,-0.395838,0.0,4.440892e-16,0.0,0.786319,0.786319,...,-0.786443,-0.373781,-0.902578,0.0,0.0,0.0,-0.786484,-0.373781,-0.902578,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,0.0,-0.786319,0.786319,-0.786319,-0.395838,0.0,4.440892e-16,0.0,0.786319,0.786319,...,-0.786443,-0.373781,-0.378592,0.0,0.0,0.0,-0.786484,-0.373781,-0.378592,0.0
40502,0.0,1.271748,-1.271748,1.271748,2.677342,0.0,4.440892e-16,0.0,-1.271748,-1.271748,...,1.271549,-0.373781,1.831974,0.0,0.0,0.0,1.271482,-0.373781,1.831974,0.0
40503,0.0,-0.786319,0.786319,-0.786319,2.677342,0.0,4.440892e-16,0.0,0.786319,0.786319,...,-0.786443,-0.373781,1.269781,0.0,0.0,0.0,-0.786484,-0.373781,1.269781,0.0
40504,0.0,1.271748,-1.271748,1.271748,-0.395838,0.0,4.440892e-16,0.0,-1.271748,-1.271748,...,1.271549,-0.373781,-0.782498,0.0,0.0,0.0,1.271482,-0.373781,-0.782498,0.0


In [82]:
X.dropna(axis=1)

,Insp. Seq No._Dam,CURE END POSITION r Collect Result_Dam,CURE END POSITION φ Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE STANDBY POSITION r Collect Result_Dam,CURE STANDBY POSITION φ Collect Result_Dam,CURE STANDBY POSITION Θ Collect Result_Dam,CURE START POSITION r Collect Result_Dam,CURE START POSITION φ Collect Result_Dam,...,Equipment_Fill1,Model.Suffix_Fill1,Workorder_Fill1,Insp Judge Code_Fill1,Wip Line_Fill2,Process Desc._Fill2,Equipment_Fill2,Model.Suffix_Fill2,Workorder_Fill2,Insp Judge Code_Fill2
0,0.0,-0.786319,0.786319,-0.786319,2.677342,0.0,4.440892e-16,0.0,0.786319,0.786319,...,-0.786443,2.860186,1.908389,0.0,0.0,0.0,-0.786484,2.860186,1.908389,0.0
1,0.0,-0.786319,0.786319,-0.786319,-0.395838,0.0,4.440892e-16,0.0,0.786319,0.786319,...,-0.786443,2.860186,-0.132974,0.0,0.0,0.0,-0.786484,2.860186,-0.132974,0.0
2,0.0,1.271748,-1.271748,1.271748,1.140752,0.0,4.440892e-16,0.0,-1.271748,-1.271748,...,1.271549,-0.373781,1.537232,0.0,0.0,0.0,1.271482,-0.373781,1.537232,0.0
3,0.0,1.271748,-1.271748,1.271748,-0.395838,0.0,4.440892e-16,0.0,-1.271748,-1.271748,...,1.271549,-0.373781,-0.307636,0.0,0.0,0.0,1.271482,-0.373781,-0.307636,0.0
4,0.0,-0.786319,0.786319,-0.786319,-0.395838,0.0,4.440892e-16,0.0,0.786319,0.786319,...,-0.786443,-0.373781,-0.902578,0.0,0.0,0.0,-0.786484,-0.373781,-0.902578,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,0.0,-0.786319,0.786319,-0.786319,-0.395838,0.0,4.440892e-16,0.0,0.786319,0.786319,...,-0.786443,-0.373781,-0.378592,0.0,0.0,0.0,-0.786484,-0.373781,-0.378592,0.0
40502,0.0,1.271748,-1.271748,1.271748,2.677342,0.0,4.440892e-16,0.0,-1.271748,-1.271748,...,1.271549,-0.373781,1.831974,0.0,0.0,0.0,1.271482,-0.373781,1.831974,0.0
40503,0.0,-0.786319,0.786319,-0.786319,2.677342,0.0,4.440892e-16,0.0,0.786319,0.786319,...,-0.786443,-0.373781,1.269781,0.0,0.0,0.0,-0.786484,-0.373781,1.269781,0.0
40504,0.0,1.271748,-1.271748,1.271748,-0.395838,0.0,4.440892e-16,0.0,-1.271748,-1.271748,...,1.271549,-0.373781,-0.782498,0.0,0.0,0.0,1.271482,-0.373781,-0.782498,0.0


In [83]:
train_data = pd.concat([X, y], axis=1)
train_data

,Insp. Seq No._Dam,CURE END POSITION r Collect Result_Dam,CURE END POSITION φ Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE STANDBY POSITION r Collect Result_Dam,CURE STANDBY POSITION φ Collect Result_Dam,CURE STANDBY POSITION Θ Collect Result_Dam,CURE START POSITION r Collect Result_Dam,CURE START POSITION φ Collect Result_Dam,...,Model.Suffix_Fill1,Workorder_Fill1,Insp Judge Code_Fill1,Wip Line_Fill2,Process Desc._Fill2,Equipment_Fill2,Model.Suffix_Fill2,Workorder_Fill2,Insp Judge Code_Fill2,target
0,0.0,-0.786319,0.786319,-0.786319,2.677342,0.0,4.440892e-16,0.0,0.786319,0.786319,...,2.860186,1.908389,0.0,0.0,0.0,-0.786484,2.860186,1.908389,0.0,Normal
1,0.0,-0.786319,0.786319,-0.786319,-0.395838,0.0,4.440892e-16,0.0,0.786319,0.786319,...,2.860186,-0.132974,0.0,0.0,0.0,-0.786484,2.860186,-0.132974,0.0,Normal
2,0.0,1.271748,-1.271748,1.271748,1.140752,0.0,4.440892e-16,0.0,-1.271748,-1.271748,...,-0.373781,1.537232,0.0,0.0,0.0,1.271482,-0.373781,1.537232,0.0,Normal
3,0.0,1.271748,-1.271748,1.271748,-0.395838,0.0,4.440892e-16,0.0,-1.271748,-1.271748,...,-0.373781,-0.307636,0.0,0.0,0.0,1.271482,-0.373781,-0.307636,0.0,Normal
4,0.0,-0.786319,0.786319,-0.786319,-0.395838,0.0,4.440892e-16,0.0,0.786319,0.786319,...,-0.373781,-0.902578,0.0,0.0,0.0,-0.786484,-0.373781,-0.902578,0.0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,0.0,-0.786319,0.786319,-0.786319,-0.395838,0.0,4.440892e-16,0.0,0.786319,0.786319,...,-0.373781,-0.378592,0.0,0.0,0.0,-0.786484,-0.373781,-0.378592,0.0,Normal
40502,0.0,1.271748,-1.271748,1.271748,2.677342,0.0,4.440892e-16,0.0,-1.271748,-1.271748,...,-0.373781,1.831974,0.0,0.0,0.0,1.271482,-0.373781,1.831974,0.0,Normal
40503,0.0,-0.786319,0.786319,-0.786319,2.677342,0.0,4.440892e-16,0.0,0.786319,0.786319,...,-0.373781,1.269781,0.0,0.0,0.0,-0.786484,-0.373781,1.269781,0.0,Normal
40504,0.0,1.271748,-1.271748,1.271748,-0.395838,0.0,4.440892e-16,0.0,-1.271748,-1.271748,...,-0.373781,-0.782498,0.0,0.0,0.0,1.271482,-0.373781,-0.782498,0.0,Normal


In [75]:
single_unique = [col for col in train_data.columns if train_data[col].nunique() == 1]
train_data = train_data.drop(single_unique, axis=1)

numeric_cols = train_data.select_dtypes(include=[int, float]).columns
concat_cols = numeric_cols.union(['target'])

train_data = train_data[concat_cols]
train_data

,1st Pressure 1st Pressure Unit Time_AutoClave,1st Pressure Collect Result_AutoClave,2nd Pressure Collect Result_AutoClave,2nd Pressure Unit Time_AutoClave,3rd Pressure Collect Result_AutoClave,3rd Pressure Unit Time_AutoClave,CURE END POSITION r Collect Result_Dam,CURE END POSITION r Collect Result_Fill2,CURE END POSITION Θ Collect Result_Dam,CURE END POSITION φ Collect Result_Dam,...,Stage3 Line3 Distance Speed Collect Result_Dam,Stage3 Line4 Distance Speed Collect Result_Dam,THICKNESS 1 Collect Result_Dam,THICKNESS 2 Collect Result_Dam,THICKNESS 3 Collect Result_Dam,Workorder_AutoClave,Workorder_Dam,Workorder_Fill1,Workorder_Fill2,target
0,-0.230459,0.908221,1.001132,-0.996596,-0.084052,-0.239759,-0.786319,-0.056565,-0.786319,0.786319,...,-0.379551,-0.383815,0.177327,0.245023,0.201131,1.908389,1.908389,1.908389,1.908389,Normal
1,-0.183893,0.721971,-0.892189,1.141562,-0.123752,-0.125808,-0.786319,-0.056565,-0.786319,0.786319,...,-1.451620,-1.453059,0.177327,0.245023,0.201131,-0.132974,-0.132974,-0.132974,-0.132974,Normal
2,-0.183893,1.466973,0.969924,-0.996596,-0.123752,-0.125808,1.271748,-0.056565,1.271748,-1.271748,...,-0.379551,-0.383815,1.430084,-0.409762,0.327059,1.537232,1.537232,1.537232,1.537232,Normal
3,-0.183893,-1.513037,-1.006621,1.141562,-0.044353,-0.125808,1.271748,-0.056565,1.271748,-1.271748,...,-1.451620,-1.453059,0.177327,0.245023,0.201131,-0.307636,-0.307636,-0.307636,-0.307636,Normal
4,-0.230459,-0.954285,-0.985815,1.123744,-0.123752,-0.239759,-0.786319,-0.059590,-0.786319,0.786319,...,1.526351,1.517063,0.177327,0.245023,0.201131,-0.902578,-0.902578,-0.902578,-0.902578,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,-0.183893,-1.326787,-1.006621,1.141562,-0.044353,-0.125808,-0.786319,-0.056565,-0.786319,0.786319,...,-0.856026,-0.859034,0.177327,0.245023,0.201131,-0.378592,-0.378592,-0.378592,-0.378592,Normal
40502,-0.230459,-0.395533,1.001132,-0.996596,-0.123752,-0.239759,1.271748,-0.056565,1.271748,-1.271748,...,-0.379551,-0.383815,-1.806206,-0.379999,-4.752049,1.831974,1.831974,1.831974,1.831974,Normal
40503,-0.183893,0.349469,0.969924,-0.996596,-0.123752,-0.125808,-0.786319,-0.056565,-0.786319,0.786319,...,-0.260432,-0.265010,0.177327,0.245023,0.201131,1.269781,1.269781,1.269781,1.269781,Normal
40504,-0.183893,-0.954285,-0.975412,1.141562,-0.084052,-0.125808,1.271748,-0.056565,1.271748,-1.271748,...,1.526351,1.517063,0.177327,0.245023,0.201131,-0.782498,-0.782498,-0.782498,-0.782498,Normal


In [76]:
# df_concat = train_data

### 언더 샘플링


데이타 불균형을 해결하기 위해 언더 샘플링을 진행합니다.


In [27]:
normal_ratio = 1.0  # 1.0 means 1:1 ratio

df_normal = train_data[train_data["target"] == "Normal"]
df_abnormal = train_data[train_data["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

  Total: Normal: 38156, AbNormal: 2350


target
AbNormal    2350
Normal      2350
Name: count, dtype: int64

### 데이터 분할


In [28]:
df_train, df_val = train_test_split(
    df_concat,
    test_size=0.2,
    stratify=df_concat["target"],
    random_state=RANDOM_STATE,
)

def print_stats(df: pd.DataFrame):
    num_normal = len(df[df["target"] == "Normal"])
    num_abnormal = len(df[df["target"] == "AbNormal"])

    print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}" + f" ratio: {num_abnormal/num_normal}")

# Print statistics
print(f"  \tAbnormal\tNormal")
print_stats(df_train)
print_stats(df_val)

  	Abnormal	Normal
  Total: Normal: 1880, AbNormal: 1880 ratio: 1.0
  Total: Normal: 470, AbNormal: 470 ratio: 1.0


## 3. 모델 학습


### 모델 정의


In [29]:
model = RandomForestClassifier(random_state=RANDOM_STATE)

### 모델 학습


In [30]:
# features = []

# for col in df_train.columns:
#     try:
#         df_train[col] = df_train[col].astype(int)
#         features.append(col)
#     except:
#         continue

train_x = df_train.drop(columns=['target'])
train_y = df_train["target"]

model.fit(train_x, train_y)

RandomForestClassifier(random_state=110)

In [31]:
test_x = df_val.drop(columns=['target'])

test_y = df_val["target"]
pred_y = model.predict(test_x)

f1 = f1_score(test_y, pred_y, pos_label='Normal')

print(f"F1 Score: {f1:.4f}")

F1 Score: 0.5671


In [11]:
pred_y

array(['Normal', 'Normal', 'Normal', ..., 'Normal', 'Normal', 'Normal'],
      dtype=object)

## 4. 제출하기


### 테스트 데이터 예측


테스트 데이터 불러오기


In [53]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))
THRESHOLD = len(test_data) / 2
test_data = test_data.dropna(thresh=THRESHOLD, axis=1)

l = ['Dam', 'Fill1', 'Fill2']

def selection(process):
    start_col = f'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_{process}'
    test_data[start_col] = test_data[start_col].apply(lambda x: np.nan if x == 'OK' else x)
    
    end_col = f'WorkMode Collect Result_{process}'
    
    selected_columns = test_data.loc[:, start_col:end_col]
    return selected_columns, start_col, end_col

def shifting(df, column):
    df_shifted = df.copy()
    nan_rows = df[df[column].isna()]
    
    for index in nan_rows.index:
        row = df_shifted.loc[index]
        shifted_row = row.shift(-1).fillna(pd.NA)
        df_shifted.loc[index] = shifted_row
    
    return df_shifted

for i in l:
    selected_columns_i, start_col_i, end_col_i = selection(i)
    df_shifted = shifting(selected_columns_i, start_col_i)
    test_data.loc[:, start_col_i:end_col_i] = df_shifted
    
test_data.to_csv('test_mod.csv', index=False)
test_data = pd.read_csv("test_mod.csv")

single_unique = [col for col in test_data.columns if test_data[col].nunique() == 1]
test_data = test_data.drop(single_unique, axis=1)

numeric_cols = test_data.select_dtypes(include=[int, float]).columns

test_data = test_data[numeric_cols]

model = RandomForestClassifier(random_state=RANDOM_STATE)

train_x = df_train.drop(columns=['target'])
train_y = df_train["target"]

common_cols = train_x.columns.intersection(test_data.columns)

train_x = train_x[common_cols]
test_x = test_data[common_cols]

model.fit(train_x, train_y)

pred_y = model.predict(test_x)

df_sub = pd.read_csv("submission.csv")
df_sub["target"] = pred_y

df_sub.to_csv("submission.csv", index=False)

ValueError: Length of values (40506) does not match length of index (17361)

In [ ]:
# df_test_x = test_data[features]

# for col in df_test_x.columns:
#     try:
#         df_test_x.loc[:, col] = df_test_x[col].astype(int)
#     except:
#         continue

In [ ]:
# test_pred = model.predict(df_test_x)
# test_pred

### 제출 파일 작성


In [ ]:
# # 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
# df_sub = pd.read_csv("submission.csv")
# df_sub["target"] = test_pred

# # 제출 파일 저장
# df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
